In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

Path to dataset files: /home/fahad/.cache/kagglehub/datasets/dylanjcastillo/7k-books-with-metadata/versions/3


In [ ]:
import pandas as pd

df = pd.read_csv(f"{path}/books.csv")
df.head()
df.to_csv('../artifacts/datasets/books.csv',index=False)

(6810, 12)

In [1]:
##### Most of the exploration is done in data wrangler

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,6))
ax = plt.axes()
sns.heatmap(df.isna().transpose(),ax=ax)
plt.title("Missing values in dataset")
plt.xlabel('Columns')
plt.ylabel('Missing Values')
plt.show()

In [ ]:
import numpy as np

df['is_missing_description'] = np.where(df['description'].isna(),1,0)
df['age_of_book'] = 2025 - df['published_year']

In [ ]:
cols_of_intrest = ['is_missing_description','age_of_book','num_pages','average_rating','ratings_count']

correlation_matrix = df[cols_of_intrest].corr(method='spearman')

ax = plt.axes()
sns.heatmap(correlation_matrix,annot=True,fmt='.2f',ax=ax,cmap='coolwarm',cbar_kws={'label':'Spearman Correlation Coefficient'})
sns.set_theme(style='white')
plt.title("Correlation Matrix")
plt.figure(figsize=(12,6))
plt.show()

In [ ]:
df = df[
    ~(df['description'].isna()) &
    ~(df['num_pages'].isna()) &
    ~(df['average_rating'].isna()) &
    ~(df['published_year'].isna()) &
    ~(df['ratings_count'].isna()) 
]

In [ ]:
df.categories.value_counts()[:20].plot(kind='bar',figsize=(12,6))

In [ ]:
df['description_len'] = df.description.apply(lambda x: len(x.split()))
df.description_len.describe()

In [ ]:
df.description_len.value_counts().sort_index().reset_index().plot(y='count',x='description_len',figsize=(12,6))

In [ ]:
df['desc_above_25_words'] = df.description_len.between(25,400).astype(int)
desc_above_25_words = df[df.desc_above_25_words==1]
len(desc_above_25_words)

In [ ]:
desc_above_25_words['title and subtitle'] = np.where(
    desc_above_25_words['subtitle'].isna(),
    desc_above_25_words['title'],
    desc_above_25_words[['title','subtitle']].astype(str).agg(': '.join,axis=1)
)

In [ ]:
desc_above_25_words['tagged_description'] = desc_above_25_words[['isbn13','description']].astype(str).agg(' '.join,axis=1)

cleaned_df = desc_above_25_words.drop(columns=['subtitle','is_missing_description','age_of_book','description_len','desc_above_25_words'])

In [ ]:
cleaned_df.shape

In [ ]:
cleaned_df.to_csv('../artifacts/datasets/cleaned.csv',index=False)